In [1]:
def get_test_config_details(lst, dataset_condition):
    ckpt_name_list = lst[1].split('/')
    train, mic_idx = ckpt_name_list[-3], ckpt_name_list[-1][-2]

    exp_name_list = lst[-1].split('_')
    #idx = 4 if exp_name_list[-1][0]=='-' else 3
    if dataset_condition=="noisy_reverb":
        snr, t60, test = exp_name_list[-1][:-3], exp_name_list[-3], exp_name_list[-5]
    elif dataset_condition=="reverb":
        snr, t60, test = None, exp_name_list[-1][:-1], exp_name_list[-3]
    elif dataset_condition=="noisy":
        snr, t60, test = exp_name_list[-1][:-3], None, exp_name_list[-3]
    else:
        pass

    return mic_idx, train, test, t60, snr

In [2]:
# getting metrics for noisy_reverb
import os
from collections import namedtuple

metrics = namedtuple('metrics', ["mix_stoi", "enh_stoi", "mix_pesq_nb", "enh_pesq_nb"])
doa_metrics = namedtuple('doa_metrics', ["mix_frm_Acc", "est_frm_Acc", "mix_blk_Acc", "est_blk_Acc"])

dataset_condition = "noisy"
comparison_domain = "doa"

scenarios = ["stationary", "moving"]
#t60_list = [0.2, 0.4, 0.6, 0.8, 1.0] # rows 
#snr_list = [-5.0, 0.0, 5.0, 10.0, 20.0] # col

files_list = os.listdir(f'../Logs/{dataset_condition}/{comparison_domain}')
exp_dict = {}
doa_exp_dict = {}
for file in files_list:
    if os.path.isfile(os.path.join(f'../Logs/{dataset_condition}/{comparison_domain}', file)) and os.path.splitext(file)[1]==".out":
        with open(os.path.join(f'../Logs/{dataset_condition}/{comparison_domain}', file), 'r') as f:
            for line in f.readlines():
                if "ckpt_dir" in line.strip():
                    mic_idx, train, test, t60, snr = get_test_config_details(line.split(), dataset_condition)
                elif "PESQ_NB_epoch" in line.strip():
                    if "PESQ_NB_epoch"==line.strip().split()[1]:
                        pesq_nb_enh = line.strip().split()[3]
                        #print(pesq_nb_enh)
                    elif "MIX_PESQ_NB_epoch"==line.strip().split()[1]:
                        pesq_nb_mix = line.strip().split()[3]
                        #print(pesq_nb_mix)
                    else:
                        continue
                    #print(line)
                elif "STOI_epoch" in line.strip():
                    if "STOI_epoch"==line.strip().split()[1]:
                        stoi_enh = line.strip().split()[3]
                        #print(stoi_enh)
                    elif "MIX_STOI_epoch"==line.strip().split()[1]:
                        stoi_mix = line.strip().split()[3]
                        #print(stoi_mix)
                    else:
                        continue
                elif "blk_Acc_epoch" in line.strip():
                    if "est_blk_Acc_epoch"==line.strip().split()[1]:
                        est_blk_Acc = line.strip().split()[3]
                    elif "mix_blk_Acc_epoch"==line.strip().split()[1]:
                        mix_blk_Acc = line.strip().split()[3]
                    else:
                        continue
                elif "frm_Acc_epoch" in line.strip():
                    if "est_frm_Acc_epoch"==line.strip().split()[1]:
                        est_frm_Acc = line.strip().split()[3]
                    elif "mix_frm_Acc_epoch"==line.strip().split()[1]:
                        mix_frm_Acc = line.strip().split()[3]
                    else:
                        continue
                
                #print(line)

    
        
        if comparison_domain =="doa":
            exp_metrics = doa_metrics(mix_frm_Acc, est_frm_Acc, mix_blk_Acc, est_blk_Acc)
        else:
            exp_metrics = metrics(stoi_mix, stoi_enh, pesq_nb_mix, pesq_nb_enh)

        #exp_metrics = { "mix_stoi": stoi_mix, "enh_stoi": stoi_enh, "mix_pesq_nb": pesq_nb_mix, "enh_pesq_nb": pesq_nb_enh}

        if dataset_condition =="reverb":
            exp_app_str = f't60_{t60}'
        elif dataset_condition =="noisy":
            exp_app_str = f'snr_{snr}'
        elif dataset_condition =="noisy_reverb":
            exp_app_str = f't60_{t60}_snr_{snr}'
        else:
            exp_app_str = ''

        key_str = f'train_{train}_mic_idx_{mic_idx}_test_{test}_{exp_app_str}' if comparison_domain!="doa" else f'train_{train}_test_{test}_{exp_app_str}'
        exp_dict[key_str] = exp_metrics
        #doa_exp_dict[f'train_{train}_mic_idx_{mic_idx}_test_{test}_{exp_app_str}'] = doa_metrics

        #break




In [5]:

import torch
torch.save(exp_dict, f"{dataset_condition}_doa_results.pt")


In [3]:
mic_idx=0
#dataset_condition = "reverb"
train="stationary"
test = "stationary"
t60=0.8
snr = -5.0
if dataset_condition =="reverb":
    exp_app_str = f't60_{t60}'
elif dataset_condition =="noisy":
    exp_app_str = f'snr_{snr}'
elif dataset_condition =="noisy_reverb":
    exp_app_str = f't60_{t60}_snr_{snr}'
else:
    exp_app_str = ''

key_str = f'train_{train}_mic_idx_{mic_idx}_test_{test}_{exp_app_str}' if comparison_domain!="doa" else f'train_{train}_test_{test}_{exp_app_str}'
exp_dict[key_str]

doa_metrics(mix_frm_Acc='0.15387825667858124', est_frm_Acc='0.4100947976112366', mix_blk_Acc='0.18034960329532623', est_blk_Acc='0.7193621397018433')

In [4]:
exp_dict

{'train_moving_test_stationary_snr_0.0': doa_metrics(mix_frm_Acc='0.27210357785224915', est_frm_Acc='0.555447518825531', mix_blk_Acc='0.39209479093551636', est_blk_Acc='0.8997735381126404'),
 'train_moving_test_moving_snr_-2.5': doa_metrics(mix_frm_Acc='0.19740110635757446', est_frm_Acc='0.49139314889907837', mix_blk_Acc='0.2623746991157532', est_blk_Acc='0.8549056649208069'),
 'train_moving_test_stationary_snr_15.0': doa_metrics(mix_frm_Acc='0.6917834281921387', est_frm_Acc='0.7959786653518677', mix_blk_Acc='0.9208812117576599', est_blk_Acc='0.976082980632782'),
 'train_stationary_test_moving_snr_-2.5': doa_metrics(mix_frm_Acc='0.19740110635757446', est_frm_Acc='0.4283803105354309', mix_blk_Acc='0.2623746991157532', est_blk_Acc='0.7674936652183533'),
 'train_stationary_test_stationary_snr_20.0': doa_metrics(mix_frm_Acc='0.7873621582984924', est_frm_Acc='0.8323114514350891', mix_blk_Acc='0.9596425294876099', est_blk_Acc='0.9806771278381348'),
 'train_moving_test_moving_snr_15.0': doa_m

In [6]:
len(exp_dict)

32

In [1]:
import torch
dataset_condition = "noisy"
comparison_domain="se"
exp_dict = torch.load(f'../Logs/{dataset_condition}/{dataset_condition}_{comparison_domain}_results.pt')

AttributeError: Can't get attribute 'metrics' on <module '__main__'>

In [2]:
#MIMO

# getting metrics for noisy_reverb
import os
from collections import namedtuple

metrics = namedtuple('metrics', ["mix_stoi", "enh_stoi", "mix_pesq_nb", "enh_pesq_nb", "mix_frm_Acc", "est_frm_Acc", "mix_blk_Acc", "est_blk_Acc"])


dataset_condition = "noisy_reverb"
comparison_domain = "doa"

scenarios = ["stationary", "moving"]
#t60_list = [0.2, 0.4, 0.6, 0.8, 1.0] # rows 
#snr_list = [-5.0, 0.0, 5.0, 10.0, 20.0] # col

files_list = os.listdir(f'../Logs/{dataset_condition}/mimo')
exp_dict = {}
doa_exp_dict = {}
for file in files_list:
    if os.path.isfile(os.path.join(f'../Logs/{dataset_condition}/mimo', file)) and os.path.splitext(file)[1]==".out":
        with open(os.path.join(f'../Logs/{dataset_condition}/mimo', file), 'r') as f:
            for line in f.readlines():
                if "ckpt_dir" in line.strip():
                    mic_idx, train, test, t60, snr = get_test_config_details(line.split(), dataset_condition)
                elif "PESQ_NB_epoch" in line.strip():
                    if "PESQ_NB_epoch"==line.strip().split()[1]:
                        pesq_nb_enh = line.strip().split()[3]
                        #print(pesq_nb_enh)
                    elif "MIX_PESQ_NB_epoch"==line.strip().split()[1]:
                        pesq_nb_mix = line.strip().split()[3]
                        #print(pesq_nb_mix)
                    else:
                        continue
                    #print(line)
                elif "STOI_epoch" in line.strip():
                    if "STOI_epoch"==line.strip().split()[1]:
                        stoi_enh = line.strip().split()[3]
                        #print(stoi_enh)
                    elif "MIX_STOI_epoch"==line.strip().split()[1]:
                        stoi_mix = line.strip().split()[3]
                        #print(stoi_mix)
                    else:
                        continue
                elif "blk_Acc_epoch" in line.strip():
                    if "est_blk_Acc_epoch"==line.strip().split()[1]:
                        est_blk_Acc = line.strip().split()[3]
                    elif "mix_blk_Acc_epoch"==line.strip().split()[1]:
                        mix_blk_Acc = line.strip().split()[3]
                    else:
                        continue
                elif "frm_Acc_epoch" in line.strip():
                    if "est_frm_Acc_epoch"==line.strip().split()[1]:
                        est_frm_Acc = line.strip().split()[3]
                    elif "mix_frm_Acc_epoch"==line.strip().split()[1]:
                        mix_frm_Acc = line.strip().split()[3]
                    else:
                        continue
                
                #print(line)

    
        

        exp_metrics = metrics(stoi_mix, stoi_enh, pesq_nb_mix, pesq_nb_enh, mix_frm_Acc, est_frm_Acc, mix_blk_Acc, est_blk_Acc)

        #exp_metrics = { "mix_stoi": stoi_mix, "enh_stoi": stoi_enh, "mix_pesq_nb": pesq_nb_mix, "enh_pesq_nb": pesq_nb_enh}

        if dataset_condition =="reverb":
            exp_app_str = f't60_{t60}'
        elif dataset_condition =="noisy":
            exp_app_str = f'snr_{snr}'
        elif dataset_condition =="noisy_reverb":
            exp_app_str = f't60_{t60}_snr_{snr}'
        else:
            exp_app_str = ''

        key_str = f'train_{train}_test_{test}_{exp_app_str}' 
        exp_dict[key_str] = exp_metrics
        #doa_exp_dict[f'train_{train}_mic_idx_{mic_idx}_test_{test}_{exp_app_str}'] = doa_metrics

        #break




In [3]:
exp_dict

{'train_stationary_test_stationary_t60_0.8_snr_0.0': metrics(mix_stoi='0.5678197160888757', enh_stoi='0.8237930815488047', mix_pesq_nb='1.3424806104946816', enh_pesq_nb='2.251976325369904', mix_frm_Acc='0.14404839277267456', est_frm_Acc='0.42789226770401', mix_blk_Acc='0.17921335995197296', est_blk_Acc='0.7054809331893921'),
 'train_stationary_test_stationary_t60_0.8_snr_20.0': metrics(mix_stoi='0.7179071205927772', enh_stoi='0.9222700608162427', mix_pesq_nb='1.8176167043343183', enh_pesq_nb='2.8290046109551907', mix_frm_Acc='0.2246537059545517', est_frm_Acc='0.5416213870048523', mix_blk_Acc='0.29116711020469666', est_blk_Acc='0.8157580494880676'),
 'train_stationary_test_moving_t60_0.4_snr_0.0': metrics(mix_stoi='0.6411536643682065', enh_stoi='0.861915714183658', mix_pesq_nb='1.4941485943507182', enh_pesq_nb='2.490244897146491', mix_frm_Acc='0.16722916066646576', est_frm_Acc='0.32033368945121765', mix_blk_Acc='0.2333357185125351', est_blk_Acc='0.5338303446769714'),
 'train_moving_test

In [5]:
import torch
torch.save(exp_dict, f"{dataset_condition}_mimo_results.pt")

In [4]:
len(exp_dict)

100